Setting the environment

드라이브 마운트

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


mecab 토크나이저 설치 및 필요 library import \\
https://pypi.org/project/python-mecab-ko/

In [2]:
!pip install python-mecab-ko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 20.9 MB/s eta 0:00:00


In [3]:
import json
import re
from mecab import MeCab
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

mecab = MeCab()

MeCab pos 종류 \\
http://openuiz.blogspot.com/2016/07/mecab-ko-dic.html

word2vec 과제에서는 mecab.nouns 사용 예정 \\
아래는 MeCab의 형태소 분류 예시이다.

In [5]:
mecab.pos("파이썬에는 참 다양한 라이브러리들이 있다. 너무 많아서 헷갈린다. 우와!")

[('파이썬', 'NNP'),
 ('에', 'JKB'),
 ('는', 'JX'),
 ('참', 'MAG'),
 ('다양', 'XR'),
 ('한', 'XSA+ETM'),
 ('라이브러리', 'NNG'),
 ('들', 'XSN'),
 ('이', 'JKS'),
 ('있', 'VA'),
 ('다', 'EF'),
 ('.', 'SF'),
 ('너무', 'MAG'),
 ('많', 'VA'),
 ('아서', 'EC'),
 ('헷갈린다', 'VV+EF'),
 ('.', 'SF'),
 ('우와', 'IC'),
 ('!', 'SF')]

Defining needed functions (+ testing)

In [4]:
# JSON 파일 로드
def load_data(json_file_path):
  try:
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data
  except: pass

Tokenizing

In [5]:
sentence = "파이썬에는 참 다양한 라이브러리들이 있다. 너무 많아서 헷갈린다. 우와!"

In [6]:
tokens = mecab.nouns(sentence)
tokens

['파이썬', '라이브러리']

json 파일에서 tokenize function 만들기

In [10]:
data = load_data('./drive/MyDrive/Word2Vec_PBL/NIKL_SPOKEN/SARW1800000001.json')

forms = [utterance['form'] for entry in data['document'] for utterance in entry['utterance']]
for i in range(20):
  print(forms[i])

요즘처럼 추운 날씨에는
따뜻한 라테 한잔 찾는 분들
많으실 텐데요.
라테 위에 그려진
다양한 라테 아트를 구경하는 것도
또 다른 재미입니다.
그런데 최근에는
라테아트를 넘어
차가운 커피 위에
색색의 크림으로
그림을 그리는
크리마트가 인기라고 하는데요
이 크리마트를 개발한 바리스타 청년을
오늘 꿈을 잡아라에서 소개합니다.
커피 중에서도
부드러운 라테는
우유와의 궁합을 자랑하는
대표적 겨울 음룐데요
특히 라테 위에 그려지는
라테아트는 눈으로 먼저


In [11]:
def filter_tokens(data):
  tokenized_sentence = []
  text_list = [utterance['form'] for entry in data['document'] for utterance in entry['utterance']]
  for item in text_list:
    # 괄호에 포함된 내용 제거
    sentence = re.sub(r'\([^)]*\)', '', item)

    tokenized_sentence.append(mecab.nouns(sentence))

  return tokenized_sentence

In [19]:
filtered = filter_tokens(data)
texts_and_tags = [(text, [f"str_{i}",]) for i, text in enumerate(filtered)]

print(filtered[:5])
texts_and_tags[:6]



[['요즘', '날씨'], ['라테', '한잔', '분'], ['텐데요'], ['라테', '위'], ['라테', '아트', '구경', '것']]


[(['요즘', '날씨'], ['str_0']),
 (['라테', '한잔', '분'], ['str_1']),
 (['텐데요'], ['str_2']),
 (['라테', '위'], ['str_3']),
 (['라테', '아트', '구경', '것'], ['str_4']),
 (['재미'], ['str_5'])]

In [22]:
TRAIN_documents = [TaggedDocument(words=text, tags=tags) for text, tags in texts_and_tags]
TRAIN_documents[:5]

[TaggedDocument(words=['요즘', '날씨'], tags=['str_0']),
 TaggedDocument(words=['라테', '한잔', '분'], tags=['str_1']),
 TaggedDocument(words=['텐데요'], tags=['str_2']),
 TaggedDocument(words=['라테', '위'], tags=['str_3']),
 TaggedDocument(words=['라테', '아트', '구경', '것'], tags=['str_4'])]

In [27]:
# 하나의 list로 data 준비
processed_data = []

# 폴더 내에서 파일 간의 tokenize를 진행
def cir_execute(file_paths):
  for file_path in file_paths:
    data = load_data(file_path)
    for item_list in filter_tokens(data):
      processed_data.append(item_list)

In [28]:
import os

def get_file_names(folder_path):
  file_names = []
  for filename in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, filename)) and filename.endswith('.json'):
      name = './drive/MyDrive/Word2Vec_PBL/NIKL_SPOKEN/' + filename
      file_names.append(name)
  return file_names

folder_path = './drive/MyDrive/Word2Vec_PBL/NIKL_SPOKEN'
file_names = get_file_names(folder_path)

In [29]:
for i in range(3):
  print(file_names[i])

print(len(file_names))

./drive/MyDrive/Word2Vec_PBL/NIKL_SPOKEN/SERW1900007025.json
./drive/MyDrive/Word2Vec_PBL/NIKL_SPOKEN/SERW1900007036.json
./drive/MyDrive/Word2Vec_PBL/NIKL_SPOKEN/SERW1900019020.json
23520


In [ ]:
# list 초기화
processed_data = []

cir_execute(file_names)

In [ ]:
texts_and_tags = [(text, [f"str_{i}",]) for i, text in enumerate(processed_data)]
TRAIN_documents = [TaggedDocument(words=text, tags=tags) for text, tags in texts_and_tags]

In [ ]:
print(len(processed_data))
print(len(TRAIN_documents))

23520
19725451


Word2Vec_DBOW

In [ ]:
model_dbow = Doc2Vec(TRAIN_documents, vector_size=100, window=5, min_count=0, workers=4)

model_dbow.save("./drive/MyDrive/Word2Vec_PBL/w2v_dbow")

Word2Vec_skip-gram

In [ ]:
model_skg = Word2Vec(sentences=processed_data, vector_size=100, window=5, min_count=1, workers=4, sg=1)

model_skg.save("./drive/MyDrive/Word2Vec_PBL/w2v_skp-grm")

**Model Test**

CBOW model

In [ ]:
model1_result_1 = model_dbow.wv.most_similar("뉴스", topn=5)
model1_result_2 = model_dbow.wv.most_similar("라테", topn=5)

print("뉴스에 대한 결과: ", model1_result_1)
print("라테에 대한 결과: ", model1_result_2)

뉴스에 대한 결과:  [('뉴습니다', 0.6285055875778198), ('뉴슨', 0.6230104565620422), ('헤드라인', 0.5879701375961304), ('이슈', 0.5734034776687622), ('가짜', 0.5141790509223938)]
라테에 대한 결과:  [('라떼', 0.8675056099891663), ('홀스', 0.8103483319282532), ('모카', 0.8096928000450134), ('카푸치노', 0.7850455045700073), ('푸치노', 0.7770894169807434)]


Skip-gram model

In [ ]:
model2_result_1 = model_skg.wv.most_similar("뉴스", topn=5)
model2_result_2 = model_skg.wv.most_similar("라테", topn=5)

print("뉴스에 대한 결과: ", model2_result_1)
print("라테에 대한 결과: ", model2_result_2)

뉴스에 대한 결과:  [('뉴습니다', 0.7894576787948608), ('뉴슨', 0.7758339047431946), ('차미연', 0.7615022659301758), ('이덕영', 0.7036344408988953), ('남효정', 0.6920825242996216)]
라테에 대한 결과:  [('푸치노', 0.8421300649642944), ('라떼', 0.8400842547416687), ('휘핑크림', 0.8169915676116943), ('홀스', 0.8155151605606079), ('얼티밋', 0.8081234693527222)]


In [ ]:
loaded_dbow = Doc2Vec.load("./drive/MyDrive/Word2Vec_PBL/w2v_dbow")
loaded_skg = Word2Vec.load("./drive/MyDrive/Word2Vec_PBL/w2v_skp-grm")

In [ ]:
!python -m gensim.scripts.word2vec2tensor --input loaded_dbow --output loaded_dbow
!python -m gensim.scripts.word2vec2tensor --input loaded_skg --output loaded_skg